In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [20]:
df = pd.read_csv("penguins.csv")

df = df.fillna(df.median())

# One-hot encode categorical features
cat_columns = df.select_dtypes(include=['object']).columns
df_encoded = pd.get_dummies(df, columns=cat_columns)

C:\Users\bsilver3\AppData\Local\Temp\ipykernel_23244\3404073856.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df.fillna(df.median())


In [23]:
df_encoded.corr()["Body Mass (g)"].nlargest(50)

Body Mass (g)                                1.000000
Flipper Length (mm)                          0.871221
Species_Gentoo penguin (Pygoscelis papua)    0.815005
Island_Biscoe                                0.625432
Culmen Length (mm)                           0.594925
Sex_MALE                                     0.421879
Date Egg_11/27/07                            0.249049
Date Egg_11/29/07                            0.182912
Date Egg_12/1/09                             0.165891
Individual ID_N56A2                          0.165112
Date Egg_11/4/08                             0.162482
Individual ID_N39A2                          0.162384
Date Egg_11/25/09                            0.161142
Date Egg_11/22/09                            0.144450
Individual ID_N32A2                          0.136934
Date Egg_11/3/08                             0.130264
Clutch Completion_Yes                        0.127328
Individual ID_N36A2                          0.125188
Individual ID_N19A2         

In [22]:
# Define Node class for decision tree
class Node:
    def __init__(self, feature_index=None, threshold=None, value=None, left=None, right=None):
        self.feature_index = feature_index  # Index of feature to split on
        self.threshold = threshold  # Threshold value to split the feature
        self.value = value  # Value to return if node is a leaf node
        self.left = left  # Left child (less than threshold)
        self.right = right  # Right child (greater than or equal to threshold)


# Define Decision Tree Regression class
class DecisionTreeRegressor:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.root = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        n_samples, n_features = X.shape
        variance = np.var(y)

        if depth == self.max_depth or n_samples == 1 or variance == 0:
            return Node(value=np.mean(y))

        best_variance_reduction = 0
        best_feature_index = None
        best_threshold = None

        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                left_indices = np.where(X[:, feature_index] < threshold)
                right_indices = np.where(X[:, feature_index] >= threshold)

                left_variance = np.var(y[left_indices])
                right_variance = np.var(y[right_indices])

                reduction = variance - (left_variance + right_variance)
                if reduction > best_variance_reduction:
                    best_variance_reduction = reduction
                    best_feature_index = feature_index
                    best_threshold = threshold

        if best_variance_reduction == 0:
            return Node(value=np.mean(y))

        left_indices = np.where(X[:, best_feature_index] < best_threshold)
        right_indices = np.where(X[:, best_feature_index] >= best_threshold)

        left_child = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_child = self._build_tree(X[right_indices], y[right_indices], depth + 1)

        return Node(feature_index=best_feature_index, threshold=best_threshold, left=left_child, right=right_child)

    def print_root_condition(self):
        print("Root condition:")
        print(f"Feature: {feature_names[self.root.feature_index]}")
        print(f"Threshold: {self.root.threshold}")

X = df_encoded.drop(columns=['Body Mass (g)']).values
y = df_encoded['Body Mass (g)'].values
feature_names = list(df_encoded.drop(columns=['Body Mass (g)']).columns)

# Create and fit decision tree regressor
tree_regressor = DecisionTreeRegressor(max_depth=3)
tree_regressor.fit(X, y)

# Visualize the decision tree
tree_regressor.print_root_condition()

Root condition:
Feature: Species_Gentoo penguin (Pygoscelis papua)
Threshold: 1.0


c:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3723: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
c:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
